#SSD-Tensorflow
This notebook provides the training instructions for SSD-Tensorflow. Using two different type of pre-trained weights. Each section contains the instructions to setup corrrectly the net, the dataset was prepared in tfrecord format.

Make sure that Colab runtime is setted up with GPU.

Remember that Google Colaboratory provides a maximum running time of twelve hours for a single training section, then the training has to bo restarted.

Further informations about SSD-Tensorflow are available at: https://github.com/balancap/SSD-Tensorflow

#1. Repository Setup
As for the others notebooks of this repository, we download all materials in the "/home/" directory.

Firstly we clone SSD-Tensorflow repository.

In [ ]:
%cd /home/
!git clone https://github.com/balancap/SSD-Tensorflow.git

Almost all SSD-Tensorflow files use "contrib", a command that is no longer available in Tensorflow 2.x, so we need to force Colab to use the 1.x version of it.

In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

After that we install all requirements, in order to use all models needed in SSD-Tensorflow repository.

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install git+https://github.com/google-research/tf-slim

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

import os
os.environ['PYTHONPATH'] += ":/content/models"

import sys
sys.path.append("/content/models")

#2. Pre-Trained Weights
In this section we download and unzip pre-treined files. Not all of this models are request, you have to chose one at least.
SSD-Tensoflow repository is provided with "ssd_300" pre-treined model, so for that we have only to unzip it. 

ssd_300

In [ ]:
%cd /home/SSD-Tensorflow/checkpoints/
!unzip ssd_300_vgg.ckpt.zip

ssd_512

In [ ]:
%cd /home/SSD-Tensorflow/checkpoints/
!gdown --id 0B0qPCUZ-3YwWT1RCLVZNN3RTVEU
!unzip VGG_VOC0712_SSD_512x512_ft_iter_120000.ckpt.zip

#3. dataset
SSD-Tensorflow is setup to run PascalVOC format dataset. We give to the net a tfrecord format converted from a KITTI format, in order to run it we need to upgrade SSD tensorflow. In the first part of this section we add new files that are able to recognize KITTI dataset.

In [ ]:
%cd /home/SSD-Tensorflow/datasets/
%rm dataset_factory.py
%rm dataset_utils.py

In [ ]:
!gdown --id 15nddpLEe_DwqFeNhrVMIsKF8lcYADl6g
!gdown --id 15nIPB3AMr591nfDCe6hIfAUXlxWhMs6Z
!gdown --id 1mYA9BQoYOJdfx7JT4h1DRfd5NhYPk3mC
!gdown --id 1Lo9TWGrgkkFWJe7hx1NobgsZowkhNXOZ

In the second part, we create a new folder inside the repository where we will download and unzip the dataset. 

In [ ]:
%cd /home/SSD-Tensorflow/
!mkdir /home/SSD-Tensorflow/data
%cd ./data/
!gdown --id 1NrDwpC0XB8-V7by-sB2DEJbnBoBgTpk6
!unzip kitti_cones.zip

#4. Weights Log
The trained weights will be saved in a folder created inside the home of your Google Drive disk. For that purpose, the notebook mount your Google Drive disk. In order to do it, you have to click the link under this first instruction and fill your Drive code to the notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir /content/drive/MyDrive/SSD-Tensorflow_logs

Lastly we clean up the GPU cache. We are now ready to start the training.

In [ ]:
!python -c "import torch; torch.cuda.empty_cache()"

#5. Training
These two blocks include the instructions to start the training.

IMPORTANT: you have to run only the correct one, depending on the pre-treined chosen weights.
If "out of memory" error occurs, try to change "batch_size" with a lower value.

If Google Colaboratory stops your run before the training has finished, you have to restart the training starting from the last weights. In order to do that, you have to replace line 9 with "--checkpoint_path=../../content/drive/MyDrive/SSD-Tensorflow_logs/model.ckpt \ ", for this you have to have your Google Drive disk still mounted.

ssd-300

In [ ]:
%cd /home/SSD-Tensorflow/

!python train_ssd_network.py \
    --train_dir=../../content/drive/MyDrive/SSD-Tensorflow_logs \
    --dataset_dir=./data/kitti_cones/tfrecord/ \
    --dataset_name=kitti \
    --dataset_split_name=train \
    --model_name=ssd_300_vgg \
    --checkpoint_path=./checkpoints/ssd_300_vgg.ckpt \
    --save_summaries_secs=60 \
    --save_interval_secs=600 \
    --weight_decay=0.0005 \
    --optimizer=adam \
    --learning_rate=0.001 \
    --batch_size=32

ssd-512

In [ ]:
%cd /home/SSD-Tensorflow/

!python train_ssd_network.py \
    --train_dir=../../content/drive/MyDrive/SSD-Tensorflow_logs \
    --dataset_dir=./data/kitti_cones/tfrecord/ \
    --dataset_name=kitti \
    --dataset_split_name=train \
    --model_name=ssd_512_vgg \
    --checkpoint_path=./checkpoints/VGG_VOC0712_SSD_512x512_ft_iter_120000.ckpt \
    --save_summaries_secs=60 \
    --save_interval_secs=600 \
    --weight_decay=0.0005 \
    --optimizer=adam \
    --learning_rate=0.001 \
    --batch_size=32